In [97]:
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', None)

In [98]:
data = pd.read_csv ('../files/HR_RAW_DATA.csv', index_col = 0)

In [99]:
data.columns

Index(['Age', 'Attrition', 'BusinessTravel', 'DailyRate', 'Department',
       'DistanceFromHome', 'Education', 'EducationField', 'employeecount',
       'employeenumber', 'EnvironmentSatisfaction', 'Gender', 'HourlyRate',
       'JobInvolvement', 'JobLevel', 'JobRole', 'JobSatisfaction',
       'MaritalStatus', 'MonthlyIncome', 'MonthlyRate', 'NUMCOMPANIESWORKED',
       'Over18', 'OverTime', 'PercentSalaryHike', 'PerformanceRating',
       'RelationshipSatisfaction', 'StandardHours', 'StockOptionLevel',
       'TOTALWORKINGYEARS', 'TrainingTimesLastYear', 'WORKLIFEBALANCE',
       'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion',
       'YEARSWITHCURRMANAGER', 'SameAsMonthlyIncome', 'DateBirth', 'Salary',
       'RoleDepartament', 'NUMBERCHILDREN', 'RemoteWork'],
      dtype='object')

In [100]:
# No se ha incluido la columna 'SameAsMonthlyIncome' ya que es exactamente igual a MonthlyIncome
salary_details = data.loc[: , ['employeenumber' , 'DailyRate', 'HourlyRate' , 'MonthlyRate' , 'MonthlyIncome' , 'PercentSalaryHike' , 'StockOptionLevel']]

### Revisar duplicados:

In [101]:
# Asigna a una variable aquellas filas de la columna employeenumber que no son null
employee_numbers_notnull = salary_details.loc[salary_details['employeenumber'].notnull() , 'employeenumber']

# Revisa si existen duplicados en los números de empleado que no son null
total_duplicados_ID = employee_numbers_notnull.duplicated().sum()
print(f"Hay {total_duplicados_ID} duplicados")

# Calcula el % de duplicados que hay en los employeenumber que no son null

print(f"El porcentaje respecto al total de IDs no nulas es: {round(total_duplicados_ID/len(employee_numbers_notnull)*100 , 2)}")

Hay 104 duplicados
El porcentaje respecto al total de IDs no nulas es: 8.79


In [102]:
# Itera por cada dato y su respectivo booleano
for dato, bool in zip(employee_numbers_notnull , employee_numbers_notnull.duplicated()):
    
    # Si el booleano es True (el dato es un duplicado) imprime la información
    if bool is True:
        print(f"----- Revisando empleado num {dato} -----")
        print(f"{data.loc[data['employeenumber'] == f'{dato}' , :].to_string(index=False)}\n")  # El apartado "to_string" ayuda a mejorar la visualización del output, ya que convierte el DataFrame de pandas en una representación de cadena de texto formateada
        
# NOTA IMPORTANTE: Los empleados coinciden en todo menos algunos en en Remote Work

----- Revisando empleado num 1541,0 -----
Age Attrition BusinessTravel DailyRate Department  DistanceFromHome  Education   EducationField  employeecount employeenumber  EnvironmentSatisfaction  Gender HourlyRate  JobInvolvement  JobLevel                JobRole  JobSatisfaction MaritalStatus MonthlyIncome  MonthlyRate  NUMCOMPANIESWORKED Over18 OverTime  PercentSalaryHike PerformanceRating  RelationshipSatisfaction StandardHours  StockOptionLevel TOTALWORKINGYEARS  TrainingTimesLastYear WORKLIFEBALANCE  YearsAtCompany YearsInCurrentRole  YearsSinceLastPromotion  YEARSWITHCURRMANAGER SameAsMonthlyIncome  DateBirth      Salary RoleDepartament  NUMBERCHILDREN RemoteWork
 34        No  travel_rarely   1440,0$        NaN                 7          2 Technical Degree              1         1541,0                        2       0         55               3         1  SaLes rEPREsENtAtivE                 3           NaN        2308,0         4944                   0    NaN      Yes             

In [103]:
# Unificamos los criterios de la columna RemoteWork a 1 y 0 para todas sus filas:

data['RemoteWork'] = data['RemoteWork'].replace({True: 1, False: 0})
data['RemoteWork'] = data['RemoteWork'].replace({'1': 1, '0': 0})
data['RemoteWork'] = data['RemoteWork'].replace({'Yes': 1, 'No': 0})
data['RemoteWork'] = data['RemoteWork'].replace({'True': 1, 'False': 0})

In [104]:
lista_empleados_remote_distinto = []

# Itera por cada dato y su respectivo booleano
for dato, bool in zip(employee_numbers_notnull , employee_numbers_notnull.duplicated()):
    
    # Si el booleano es True (el dato es un duplicado)
    if bool is True:
        # Asigna el resultado a una variable
        employee_data = data.loc[data['employeenumber'] == f'{dato}' , :]
        
        # Revisa si hay elementos únicos en la columna RemoteWork
        remote_work_values = employee_data['RemoteWork'].unique()
        
        # Si hay más de 1 elemento único en la columna
        if len(remote_work_values) > 1:
            # Guarda en una lista aquellos que cumplen la condición (duplicados con datos distintos en RemoteWork)
            lista_empleados_remote_distinto.append(dato)

print(f"El listado de empleados duplicados y con RemoteWork diferentes en el DataFrame es:")
print(f"{lista_empleados_remote_distinto}")
print(f"De los 104 duplicados, hay un total de {len(lista_empleados_remote_distinto)} que tienen datos de RemoteWork distintos")

El listado de empleados duplicados y con RemoteWork diferentes en el DataFrame es:
['1541,0', '1947,0', '1135,0', '1157,0', '1160,0', '1161,0', '1162,0', '1778,0', '1797,0', '1869,0', '1898,0', '1911,0', '424,0', '447,0', '455,0', '460,0', '470,0', '475,0', '478,0', '482,0', '495,0', '522,0', '530,0', '376,0', '381,0', '416,0', '430,0', '446,0', '454,0', '458,0', '476,0', '488,0', '500,0', '586,0', '621,0', '663,0', '717,0', '724,0', '747,0', '762,0', '789,0', '803,0', '823,0', '842,0', '913,0', '941,0', '966,0']
De los 104 duplicados, hay un total de 47 que tienen datos de RemoteWork distintos


In [105]:
for columna in salary_details.columns:
    print(f"Los datos únicos de {columna} son:")
    print(f"{salary_details[columna].unique()}\n")
    
# Hay datos como nan$ que no captó como nulos en primera instancia por incluir $

Los datos únicos de employeenumber son:
['162,0' '259,0' '319,0' ... '2012,0' '2023,0' '2040,0']

Los datos únicos de DailyRate son:
['684,0$' '699,0$' '532,0$' '359,0$' '1319,0$' '117,0$' '1435,0$' '635,0$'
 '1276,0$' '840,0$' '247,0$' '1369,0$' '201,0$' '1360,0$' '692,0$'
 '1398,0$' '286,0$' '1402,0$' '819,0$' '884,0$' '1238,0$' '515,0$'
 '1223,0$' '202,0$' '928,0$' '607,0$' '266,0$' '429,0$' '589,0$' 'nan$'
 '1180,0$' '1282,0$' '776,0$' '665,0$' '526,0$' '1034,0$' '1403,0$'
 '1499,0$' '580,0$' '859,0$' '263,0$' '1376,0$' '885,0$' '1003,0$'
 '1321,0$' '394,0$' '1372,0$' '1333,0$' '228,0$' '737,0$' '823,0$'
 '667,0$' '301,0$' '573,0$' '1329,0$' '630,0$' '1063,0$' '1017,0$'
 '1296,0$' '939,0$' '1355,0$' '1448,0$' '200,0$' '1202,0$' '404,0$'
 '208,0$' '813,0$' '465,0$' '1189,0$' '1001,0$' '1394,0$' '161,0$'
 '288,0$' '682,0$' '1354,0$' '147,0$' '119,0$' '1413,0$' '452,0$' '334,0$'
 '1132,0$' '982,0$' '480,0$' '1099,0$' '672,0$' '1379,0$' '583,0$'
 '1492,0$' '1050,0$' '469,0$' '237,0$' '

In [112]:
data['DailyRate'] = data['DailyRate'].str.replace('$', '')

/tmp/ipykernel_23737/265929712.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  data['DailyRate'] = data['DailyRate'].str.replace('$', '')


In [111]:
data['DailyRate'].isnull()

0       False
1       False
2       False
3       False
4       False
        ...  
1609    False
1610    False
1611    False
1612    False
1613    False
Name: DailyRate, Length: 1614, dtype: bool

In [108]:
print(f"El % de nulos por cada columna es:")
print(f"{round(salary_details.isnull().sum()/salary_details.shape[0]*100 , 2)}")

print(f"\nObservaciones:")
print(f"'MonthlyIncome' demasiado elevado --> Necesitamos encontrar la manera de calcular los ingresos mensuales que faltan")
print(f"'employeenumber' demasiado elevado --> Necesitamos encontrar la manera de rellenar con IDs aquellos a los que les falten")

El % de nulos por cada columna es:
employeenumber       26.70
DailyRate             0.00
HourlyRate            0.00
MonthlyRate           0.00
MonthlyIncome        52.23
PercentSalaryHike     0.00
StockOptionLevel      0.00
dtype: float64

Observaciones:
'MonthlyIncome' demasiado elevado --> Necesitamos encontrar la manera de calcular los ingresos mensuales que faltan
'employeenumber' demasiado elevado --> Necesitamos encontrar la manera de rellenar con IDs aquellos a los que les falten
